In [ ]:
!pip install --upgrade "ibm-watson>=4.7.1"
!pip install python-telegram-bot

# Funções auxiliares

In [ ]:
def updateEntidades(output, user_id):
    for out in output['output']['entities']:
        BANCO_DADOS[user_id]['ENTIDADES'].update({out['entity']: out['value']})
    
    if len(BANCO_DADOS[user_id]['ENTIDADES']) == 4:
        return True
    return False

In [ ]:
def sugerirOculos(user_id):
    catalogo = pd.read_csv('catalogo.csv', sep=';')
    formato_oculos = {'oval': ['redondo', 'retangular', 'triangular'], 'quadrado': ['redondo', 'triangular'], 'redondo': ['quadrado', 'retangular', 'triangular']}[BANCO_DADOS[user_id]['ENTIDADES']['formato_do_rosto']]
    cor = BANCO_DADOS[user_id]['ENTIDADES']['cor']
    genero = BANCO_DADOS[user_id]['ENTIDADES']['genero']
    
    modelos = list()

    for f in formato_oculos:
        temp = catalogo.loc[(catalogo['formato_oculos'] == f) & (catalogo['genero'] == genero)]
        modelos.append(temp)

    opcao_sem_cor = pd.concat(modelos)
    opcao_com_cor = opcao_sem_cor.loc[opcao_sem_cor['cor'] == cor]

    if len(opcao_com_cor) == 0:
        return 0, opcao_sem_cor
    else:
        return len(opcao_com_cor), opcao_com_cor

# Watson

In [ ]:
ASSISTANT_APIKEY = '6SbPNEcnalIurb5LSlOnqr-DkdZBbWAZugC_JFkYFzUS'
ASSISTANT_URL = 'https://api.us-south.assistant.watson.cloud.ibm.com/instances/7f639ca4-251a-4a66-b21f-b020a9c92ce3'
ASSISTANT_ID = 'e0482367-a817-420e-b5a3-a27dd800aa0f'

from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(ASSISTANT_APIKEY)
assistant = AssistantV2(
    version='2020-04-01',
    authenticator=authenticator
)

assistant.set_service_url(ASSISTANT_URL)

In [ ]:
VISUAL_RECOGNITION_APIKEY = "w3ZEWWwtmKkhjT8cCvmb7Tl7BA8XzEFS-2kYZEhaoG8u"
VISUAL_RECOGNITION_URL = 'https://api.us-south.visual-recognition.watson.cloud.ibm.com/instances/d247f614-69ad-410e-992f-c69df43501b0'
CLASSIFIER_ID = 'DefaultCustomModel_962446588'

from ibm_watson import VisualRecognitionV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(VISUAL_RECOGNITION_APIKEY)
visual_recognition = VisualRecognitionV3(
    version='2018-03-19',
    authenticator=authenticator
)

visual_recognition.set_service_url(VISUAL_RECOGNITION_URL)

# Telegram

In [ ]:
BANCO_DADOS = dict()

APIKEY = '1462738723:AAHpUeLjo7Dgb2SOMG5ploK8-ngofpXmv9Q'

from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove, Update

import logging, time

updater = Updater(APIKEY, use_context=True)
dp = updater.dispatcher

dp.add_handler(MessageHandler(Filters.regex(r'mais'), ver_mais))
dp.add_handler(MessageHandler(Filters.regex(r'outro'), ver_mais))

dp.add_handler(MessageHandler(Filters.text, watson))
dp.add_handler(MessageHandler(Filters.voice, classificadorSom)) 
dp.add_handler(MessageHandler(Filters.photo, classificadorImagem))

updater.start_polling()
updater.idle()

User id 1108603766 diz: Oi
User id 1108603766 diz: Tchau
User id 1108603766 diz: Quero o telefone da loja
User id 1108603766 diz: Quero comprar uma lente
User id 1108603766 diz: Não, obrigado


In [ ]:
def ver_mais(bot, context):

    user = bot.message.chat.id
    print("User id {} diz: {}".format(user, bot.message.text))
    

    ## Identifica usuario
    chat_id = bot.message.chat.id

    r = sugerirOculos(chat_id)

    bot.message.reply_text('O que acha desse? Sua cara, né!')

    escolhe_aleatorio = r[1].sample().iloc[0]['codigo']

    bot.message.reply_photo('https://raw.githubusercontent.com/rhuam/datasets/master/oculos_ono/{}/img_({}).JPG'.format(escolhe_aleatorio, randint(1, 4)))
  

In [ ]:
from random import randint

def watson(bot, context):

    user = bot.message.chat.id
    print("User id {} diz: {}".format(user, bot.message.text))

    ## Identifica usuario
    chat_id = bot.message.chat.id

    if chat_id in BANCO_DADOS:
        session_id = BANCO_DADOS[chat_id]['session_id']
    else:
        response = assistant.create_session(assistant_id=ASSISTANT_ID).get_result()
        session_id = response['session_id']
        BANCO_DADOS[chat_id] = {'session_id': session_id, 'ENTIDADES': {}}

    ## Envia mensagem para Watson
    response = assistant.message(
        assistant_id=ASSISTANT_ID,
        session_id=session_id,
        input={
        'message_type': 'text',
        'text': bot.message.text
        }
    ).get_result()

    for resposta in response['output']['generic']:
        bot.message.reply_text(resposta['text'])

    if 'Tchau' in resposta['text']:
        assistant.delete_session(assistant_id=ASSISTANT_ID, session_id=session_id).get_result()
        return None

    if updateEntidades(response, chat_id):
        r = sugerirOculos(chat_id)

        if r[0] == 0:
            bot.message.reply_text('Não encontrei um óculos da cor que vocês deseja, mas olha esse:')
        else:
            bot.message.reply_text('Encontrei {} óculo(s) para você.'.format(r[0]))
    
        escolhe_aleatorio = r[1].sample().iloc[0]['codigo']
        URL = 'https://raw.githubusercontent.com/rhuam/datasets/master/oculos_ono/{}/img_({}).JPG'.format(escolhe_aleatorio, randint(1, 4))
        print(URL)
        bot.message.reply_photo(URL)


In [ ]:
import pandas as pd
import numpy as np
import librosa
from joblib import load

def classificadorSom(bot, context):
    user = bot.message.chat.id
    print("User id {} enviou um áudio".format(user))

    ## Identifica usuario
    chat_id = bot.message.chat.id

    if chat_id in BANCO_DADOS:
        session_id = BANCO_DADOS[chat_id]['session_id']
    else:
        response = assistant.create_session(assistant_id=ASSISTANT_ID).get_result()
        session_id = response['session_id']
        BANCO_DADOS[chat_id] = {'session_id': session_id}

    ## Salva o áudio
    photo_file = bot.message.voice.get_file()
    file_name = str(bot.message.voice.file_unique_id) + '.ogg'
    photo_file.download(file_name)

    ## Mensagem para aguardar
    bot.message.reply_text("Só um momento, por favor!")

    ## Modelo de ML
    modelo  = load('sound_recognition.joblib')


    ## Predição
    genero = modelo.predict(pd.DataFrame([file_name]))[0]

    print("Voz do usuário {} é compatível com gênero {}".format(user, genero))

    mensagem = 'Eu sou ' + genero

    ## Envia mensagem para Watson
    response = assistant.message(
        assistant_id=ASSISTANT_ID,
        session_id=session_id,
        input={
        'message_type': 'text',
        'text': mensagem
        }
    ).get_result()


    for resposta in response['output']['generic']:
        bot.message.reply_text(resposta['text'])

    if updateEntidades(response, chat_id):
        r = sugerirOculos(chat_id)

        if r[0] == 0:
            bot.message.reply_text('Não encontrei um óculos da cor que vocês deseja, mas olha esse:')
        else:
            bot.message.reply_text('Encontrei {} óculo(s) para você.'.format(r[0]))
    
        escolhe_aleatorio = r[1].sample().iloc[0]['codigo']
        bot.message.reply_photo('https://raw.githubusercontent.com/rhuam/datasets/master/oculos_ono/{}/img_({}).JPG'.format(escolhe_aleatorio, randint(1, 4)))

In [ ]:
import pandas as pd
import numpy as np


def classificadorImagem(bot, context):
    user = bot.message.chat.id
    print("User id {} enviou uma foto".format(user))

    ## Identifica usuario
    chat_id = bot.message.chat.id

    if chat_id in BANCO_DADOS:
        session_id = BANCO_DADOS[chat_id]['session_id']
    else:
        response = assistant.create_session(assistant_id=ASSISTANT_ID).get_result()
        session_id = response['session_id']
        BANCO_DADOS[chat_id] = {'session_id': session_id}

    ## Salva foto
    photo_file = bot.message.photo[-1].get_file()
    file_name = str(bot.message.photo[-1].file_unique_id) + '.jpg'
    photo_file.download(file_name)

    ## Mensagem para aguardar
    bot.message.reply_text("Mais um momento, por favor!")

    ## Modelo de ML
    with open(file_name, 'rb') as images_file:
        classes = visual_recognition.classify(images_file=images_file, classifier_ids=[CLASSIFIER_ID]).get_result()

    formato_rosto, score = '', 0
    for c in classes['images'][0]['classifiers'][0]['classes']:
        if c['score'] > score:
            formato_rosto = c['class']


    mensagem = "Meu rosto é " + {'Oval': 'oval', 'Square': 'quadrado', 'Round': 'redondo'}[formato_rosto]

    print("Rosto do usuário {} é compatível com formato {}".format(user, formato_rosto))


    ## Envia mensagem para Watson
    response = assistant.message(
        assistant_id=ASSISTANT_ID,
        session_id=session_id,
        input={
        'message_type': 'text',
        'text': mensagem
        }
    ).get_result()


    for resposta in response['output']['generic']:
        bot.message.reply_text(resposta['text'])

    if updateEntidades(response, chat_id):
        r = sugerirOculos(chat_id)

        if r[0] == 0:
            bot.message.reply_text('Não encontrei um óculos da cor que vocês deseja, mas olha esse:')
        else:
            bot.message.reply_text('Encontrei {} óculo(s) para você.'.format(r[0]))
    
        escolhe_aleatorio = r[1].sample().iloc[0]['codigo']
        bot.message.reply_photo('https://raw.githubusercontent.com/rhuam/datasets/master/oculos_ono/{}/img_({}).JPG'.format(escolhe_aleatorio, randint(1, 4)))

# Classe do modelo

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class AudioDecode(BaseEstimator, TransformerMixin):
    def __main__(self):
        pass
    
    def __init__(self, offset=0.5, duration=1):
        self.offset = offset
        self.duration = duration

    def fit(self, X, y = None):
        self.X = X
        return self
    
    def transform(self, X, y = None):
        X_ = list()
        for index, value in X.iterrows():
            audio_data, sr = librosa.load(value[0], offset=self.offset, duration=self.duration)
            audio_stft = librosa.stft(audio_data)
            audio_db = librosa.amplitude_to_db(abs(audio_stft))
            X_.append(np.reshape(audio_db, audio_db.size))
        
        return pd.DataFrame(X_)